# How to Deploy the ML Models

Here, the TensorFlow model is converted to TensorFlow.js.
The model fine tuned for distinguishing 20 classes is used here.

Also, a brief tutorial is included for the WebDevs on how to use the model in JavaScript.

TLDR version: The converted model can be found here: 'wt23-wastewise/AI/MLOps/tensorflow_js/model.json'
_
## Sources

This is based on the tensor flow documentation and especially this:
https://www.tensorflow.org/js/tutorials/conversion/import_saved_model

# Preparations

In [1]:
# connect colab to google drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# navigate to WasteWise Git repository and get updates
%cd drive/MyDrive/wt23-wastewise

/content/drive/MyDrive/wt23-wastewise


In [ ]:
# install libraries, packages, dependencies
!pip install tensorflowjs # tensorflow to javascript converter
!pip install pyyaml h5py  # required to save models in HDF5 format

In [ ]:
# load libraries, packages, dependencies
import tensorflow as tf
from tensorflow import keras

# Converting the Model

Use the TensorFlow.js converter to convert the model from Python to JavaScript.


In [ ]:
# optional: have a look at the model first
# this is just for getting an overview and is not necessary for the conversion itself

# load saved_model 
model = tf.keras.models.load_model('wt23-wastewise/AI/MLOps/tensorflow_js/model.json')

# get a summary of the model including information about the layers etc.
model.summary()

In [ ]:
# call the TensorFlow to JavaScript converter
# it is a shell command, hence insert the exclamation mark if called in a Python environment
!tensorflowjs_converter \
    --input_format=tf_saved_model \          # format of the exported model. we used saved model format
    --saved_model_tags=serve \               # tags of MetaGraphDef to load, comma separated
    AI/models/model_20_classes \             # positional argument: input path
    AI/MLOps/tensorflow_js   # positional argument: output path

# Deploy in JavaScript and Inference/Integrate with App

This is the previously mentioned tutorial part for the WebDevs.

The file containing the converted model that you need is: "wt23-wastewise/AI/MLOps/tensorflow_js/model.json".

# Step 1: Install the tfjs-converter npm package
This step is to be executed in the terminal.
```bash
yarn add @tensorflow/tfjs or npm install @tensorflow/tfjs
```

# Step 2: Load the dependencies

From this step one, do in JavaScript.

```javascript
import * as tf from '@tensorflow/tfjs';
import {loadGraphModel} from '@tensorflow/tfjs-converter';
```

# Step 3: Load the model

```javascript
// store absolute path to the TensorFlow.js model in variable "MODEL_URL"
const MODEL_URL = 'wt23-wastewise/AI/MLOps/tensorflow_js/model.json';

// load the model and store it as an object in variable "model"
const model = await loadGraphModel(MODEL_URL);;
```

The loaded model is now callable using "model".

# Step 4: Get the image
- Use the camera to take a photo.
- Save that photo in a variable

```javascript
// example: call the variable "waste"
// in place of "...", insert the code calling the camera
const waste = ... ; 
```


# Step 5: Query the model

Use the method "execute" to query the model for inference.

Try if this works already. There is a chance that "tf.browser.fromPixels" prepares the image in the desired way and that nothing else is necessary for this.

```javascript
// this is how you query the model: use the method "execute"
model.execute(tf.browser.fromPixels(waste));
```
The input images need to be 224 x 224 normalised pixels in the correct file format. If querying the network does not work as intended, please follow the instructions below. There is a good chance that the problems are caused by the image not being correctly formatted.

# Step 6: Prepare images for being input to the algorithm

If the instructions provided in step 5 do not just work like this, try these.

## Image file format

TensorFlow accepts jpeg, png, bmp (and gif, but that should not be relevant right here). 
I think it would be best just to use png for simlicity, but choose whatever you prefer.
Other file formats are not accepted.

## Resize and normalize the image

Required format: `img_size = (224, 224)`

What's inputted to the model must be 224 times 224 pixels, not more, not less. 

Also, the model only takes pixel values between 0 and 1.

Below, you can find a proposal for doing something similar. I adapted this from code generated by ChatGPT. Please adapt to your needs.

```javascript
// Load the png image
// Probably just use "waste" 
const image = new Image();
image.src = "path/to/image.png";

// Wait for the image to load before processing it
image.onload = function() {
  // Create a canvas element to draw the image on
  const canvas = document.createElement("canvas");
  canvas.width = 224;
  canvas.height = 224;

  // Draw the image on the canvas
  const ctx = canvas.getContext("2d");
  ctx.drawImage(image, 0, 0, 224, 224);

  // Convert the canvas to a TensorFlow.js tensor
  const tensor = tf.browser.fromPixels(canvas)
    .toFloat()
    .expandDims();

  // Normalize the tensor
  const normalized = tensor.div(255.0);

  // Use the tensor as input for your image classifier model
  // This part is likely not needed and might not even work
  // Perhaps exchange "model.predict" with "model.execute"
  const prediction = model.predict(normalized);
  // Prints out the prediction to console
  console.log(prediction);
};
```


# Step 7: Translate from waste type to waste bin

Hopefully the previous part worked and it was possible to query the model. If that worked, we should have gotten a prediction of the class. At the time of writing, the classifier is able to distinguish between 20 classes of waste. We now need a recommendation for the disposal of each class. For this, I suggest a dictionary approach:

```
These are the classes currently used in the classifier:

['aluminum_foil',
 'apples',
 'banana_peels',
 'cardboard',
 'condoms',
 'diapers',
 'food_waste',
 'glass_bottle',
 'old_books',
 'oranges',
 'pans',
 'pizza_box',
 'plastic_bags',
 'plastic_packaging',
 'plastic_toys',
 'smartphone',
 'tampons',
 'tea_bags',
 'tetrapack',
 'toothbrush']
 ```

 ```python
 # this is how it could be done in Python
 def recommend_bin(prediction):
    """
    Returns the value associated with the given key in the given dictionary.
    Dictionary keys are waste type, values are recommended waste bin. 
    Thus, recommended waste bin is returned for the waste type.
    """

    dictionary = {"banana_peel": "biological_waste", "plastic_packaging": "plastic_waste", "cardboard": "paper_waste"}
    
    return dictionary[prediction]
 ```

 ```javascript
 // here is the translation to JavaScript
 // it is just a suggestion, please adapt if there are mistakes in the translation
 function recommendBin(prediction) {
  /*
  Returns the value associated with the given key in the given dictionary.
  Dictionary keys are waste type, values are recommended waste bin.
  Thus, recommended waste bin is returned for the waste type.
  */

  const dictionary = {
    "banana_peel": "biological_waste",
    "plastic_packaging": "plastic_waste",
    "cardboard": "paper_waste"
  };

  return dictionary[prediction];
}
```

The remaining classes must be added (specified above) and the correct waste bins need to be set (see work from the UserExperience team). Subsequently, the function needs to be integrated with the rest of the pipeline.

Finally, some output needs to be made. I suggest class, recommended waste and maybe confidence. This is optional, however.

Consider adding a button for the user to label the image as misclassified.